# OPTaaS Warm Start

### <span style="color:red">Note:</span> To run this notebook, you need an API Key. You can get one <a href="mailto:info@mindfoundry.ai">here</a>.

You can "warm-start" an OPTaaS Task by providing some results that you've already calculated. This should help you get to the optimum faster.

## Connect to OPTaaS using your API Key

In [1]:
library(optaas.client)

client <- OPTaaSClient$new("https://optaas.mindfoundry.ai", "Your OPTaaS API Key")

## Define your parameters

In [2]:
parameters <- list(
    BoolParameter('my_bool'),
    CategoricalParameter('my_cat', values=list('a', 'b', 'c'), default='c'),
    ChoiceParameter('ints_or_floats', choices=list(
        GroupParameter('ints', items=list(
            IntParameter('my_int', minimum=0, maximum=20),
            IntParameter('my_optional_int', minimum=-10, maximum=10, optional=TRUE)
        )),
        GroupParameter('floats', items=list(
            FloatParameter('float1', minimum=0, maximum=1),
            FloatParameter('float2', minimum=0.5, maximum=4.5)
        ))
    ))
)

## Define your scoring function

The argument names in your scoring function must match the parameter names you defined above.

In [3]:
scoring_function <- function(my_bool, my_cat, ints_or_floats) {
    score <- if (isTRUE(my_bool)) 5 else -5
    score <- if (my_cat == 'a') score + 1 else score + 3
    if (!is.null(ints_or_floats$ints)) {
        score <- score + do.call(sum, ints_or_floats$ints)
    } else {
        score <- score * do.call(sum, ints_or_floats$floats)
    }
    score
}

## Create your Task

In [4]:
task <- client$create_task(
    title="Dummy task",
    parameters=parameters,
    goal="min",  # optional (default is "max")
    min_known_score=-22, max_known_score=44  # optional
)

## Create some warm-start configurations

In [5]:
good_configuration <- list(my_bool=FALSE, my_cat='a', ints_or_floats=list(floats=list(float1=1, float2=3)))
good_score <- do.call(scoring_function, good_configuration)

bad_configuration <- list(my_bool=TRUE, my_cat='b', ints_or_floats=list(ints=list(my_int=20)))
bad_score <- do.call(scoring_function, bad_configuration)

task$add_user_defined_configuration(configuration=good_configuration, score=good_score)
task$add_user_defined_configuration(configuration=bad_configuration, score=bad_score)

$id
[1] "495ed671-91a3-48a2-a0cc-f03f018dff56"

$`_links`
$`_links`$self
$`_links`$self$href
[1] "/api/v1/tasks/459cfd18-9c54-4ac3-aa7c-9cef8f229d0c/configurations/495ed671-91a3-48a2-a0cc-f03f018dff56"


$`_links`$results
$`_links`$results$href
[1] "/api/v1/tasks/459cfd18-9c54-4ac3-aa7c-9cef8f229d0c/configurations/495ed671-91a3-48a2-a0cc-f03f018dff56/results"



$values
$values$my_bool
[1] FALSE

$values$my_cat
[1] "a"

$values$ints_or_floats
$values$ints_or_floats$floats
$values$ints_or_floats$floats$float1
[1] 1

$values$ints_or_floats$floats$float2
[1] 3




$type
[1] "user-defined"

$id
[1] "b9f0c7f7-1884-45d3-862e-4fbe6a23707e"

$`_links`
$`_links`$self
$`_links`$self$href
[1] "/api/v1/tasks/459cfd18-9c54-4ac3-aa7c-9cef8f229d0c/configurations/b9f0c7f7-1884-45d3-862e-4fbe6a23707e"


$`_links`$results
$`_links`$results$href
[1] "/api/v1/tasks/459cfd18-9c54-4ac3-aa7c-9cef8f229d0c/configurations/b9f0c7f7-1884-45d3-862e-4fbe6a23707e/results"



$values
$values$my_bool
[1] TRUE

$values$my_cat
[1] "b"

$values$ints_or_floats
$values$ints_or_floats$ints
$values$ints_or_floats$ints$my_int
[1] 20




$type
[1] "user-defined"

## Run your Task

In [6]:
best_result <- task$run(scoring_function=scoring_function, number_of_iterations=10)

print(paste("Best Score:", best_result$score))
print("with configuration:")
print(best_result$configuration$values)

[1] "Iteration: 1   Score: 8"
[1] "Iteration: 2   Score: 8"
[1] "Iteration: 3   Score: -6"
[1] "Iteration: 4   Score: 10"
[1] "Iteration: 5   Score: 19"
[1] "Iteration: 6   Score: 7"
[1] "Iteration: 7   Score: -8.89617074090028"
[1] "Iteration: 8   Score: -3.92257123192305"
[1] "Iteration: 9   Score: 25.3918140543229"
[1] "Iteration: 10   Score: -16.8599102274374"
[1] "Best Score: -16.8599"
[1] "with configuration:"
$my_bool
[1] FALSE

$my_cat
[1] "a"

$ints_or_floats
$ints_or_floats$floats
$ints_or_floats$floats$float1
[1] 0.972055

$ints_or_floats$floats$float2
[1] 3.242923



